In [1]:
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
from ipywidgets import interact, Layout
import cvzone
import time
import threading
import multiprocessing as mp
from multiprocessing.shared_memory import SharedMemory
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import t2
import pickle, dill
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
device = (
    "cuda"
    if torch.cuda.is_available()
    # else "mps"
    # if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

cpu


In [2]:
font = cv2.FONT_HERSHEY_SIMPLEX
width_img = 1280
height_img = 960

a = [0,255,0,255,0,255]
hue = widgets.IntRangeSlider(min=0, max=258, step=1, value=[a[0], a[1]], layout=Layout(width='80%'), description='Hue')
sat = widgets.IntRangeSlider(min=0, max=255, step=1, value=[a[2], a[3]], layout=Layout(width='80%'), description='Saturation')
val = widgets.IntRangeSlider(min=0, max=255, step=1, value=[a[4], a[5]], layout=Layout(width='80%'), description='Value')

x1 = widgets.IntSlider(min=0, max=width_img, step=1, value=100, layout=Layout(width='80%'), description='x1')
y1 = widgets.IntSlider(min=0, max=height_img, step=1, value=100, layout=Layout(width='80%'), description='y1')
w = widgets.IntSlider(min=0, max=width_img, step=1, value=300, layout=Layout(width='80%'), description='w')
h = widgets.IntSlider(min=0, max=height_img, step=1, value=300, layout=Layout(width='80%'), description='h')

In [81]:
tmp = mp.Value('i', 0)
contours_size = mp.Value('i', 0)
len_contours = mp.Value('i', 0)

frame_memory = SharedMemory(name='FrameMemory', create=True, size=width_img*height_img*3)
contours_memory = SharedMemory(name='ContoursMemory', create=True, size=4096)

p1 = mp.Process(target=t2.my_process, args=(tmp, width_img, height_img, len_contours, contours_size,), daemon=True)
p1.start()

stopButton = widgets.ToggleButton(description='Stop', disabled=False)
output = widgets.HTML(value=' ', description='Train:', disabled=False)
grid = widgets.GridBox([stopButton, output], layout=widgets.Layout(grid_template_columns='repeat(2, 200px)'))

def preprocess(fr):
    mask = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
    mask = cv2.GaussianBlur(mask, (5,5), 2)
    mask = cv2.Canny(mask, 30, 255, apertureSize=5)
    return mask

def contours(fr_m, fr):
    biggest = np.array([])
    max_area = 0
    contours, hierarchy = cv2.findContours(fr_m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 100_000:
            epsilon = 0.1 * cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, epsilon, True)
            cv2.drawContours(fr, cnt, -1, (0,255,0), 5)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest

def reorder(my_points):
    my_points = my_points.reshape((4,2))
    my_points_new = np.zeros((4,1,2), np.int32)
    add = my_points.sum(1)

    my_points_new[0] = my_points[np.argmin(add)]
    my_points_new[3] = my_points[np.argmax(add)]

    diff = np.diff(my_points, axis=1)

    my_points_new[1] = my_points[np.argmin(diff)]
    my_points_new[2] = my_points[np.argmax(diff)]

    return my_points_new

def get_warp(img, approx):
    biggest = reorder(approx)

    pts1 = np.float32(biggest)
    pts2 = np.float32([[0, 0], [width_img, 0], [0, height_img], [width_img, height_img]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    img_output = cv2.warpPerspective(img, matrix, (width_img, height_img))
    img_cropped = cv2.resize(img_output, (width_img, height_img))
    image_grayworld = (img_cropped * (img_cropped.mean() / img_cropped.mean(axis=(0, 1)))).astype(np.uint8)
    return image_grayworld

# ID_card TRAIN
def text_detection(img, contours_data):
    tensor = np.empty((1,4))
    for contour in contours_data[:,:]:
        x, y, w, h = contour[0], contour[2], contour[1], contour[3]
        cv2.rectangle(img, (x, y), (w, h), (255, 255, 0), 6)
        tensor = np.append(tensor, [[x, y, w, h]], axis=0)
    if len_contours.value > 1:
        m = int(184/4)
        t = tensor[1:m,:].astype(int)
        t = np.pad(t, [(0, 46 - t.shape[0]),(0, 0)], mode='constant') # 184/4
        t = t.T.reshape(1, -1)
        x11 = torch.tensor(t).float().to(device)
        prediction = model(x11)
        a = float(torch.softmax(prediction, dim=1)[0][0])
        a = str(round(a, 3))
        b = float(torch.softmax(prediction, dim=1)[0][1])
        b = str(round(b, 3))
        c = float(torch.softmax(prediction, dim=1)[0][2])
        c = str(round(c, 3))
        d = float(torch.softmax(prediction, dim=1)[0][3])
        d = str(round(d, 3))
        output.value = a + '/' + b + '/' + c + '/' + d
        
        
        # if len(l2) < 400:   
        #     l2.append((tensor[1:,:].astype(int), 3))
        # else:
        #     output.value = 'complete'

# ID_card CHECK
# def text_detection(img, contours_data):
#     t = np.empty((1,4))
#     for contour in contours_data[:,:]:
#         x, y, w, h = contour[0], contour[2], contour[1], contour[3]
#         cv2.rectangle(img, (x, y), (w, h), (255, 255, 0), 6)
#         t = np.append(t, [[x, y, w, h]], axis=0)
    
#     if len_contours.value > 1:
#         t = t[1:,:] # 1:13
#         # t1 = np.pad(t, [(0, 12 - t.shape[0]),(0, 0)], mode='constant') # s=12
#         # x = torch.tensor(np.reshape(t1, (1,t1.shape[0]*4))).float().to(device)
#         # prediction = model(x)
#         # a = float(torch.softmax(prediction, dim=1)[0][0])
#         # a = str(round(a, 3))
#         # b = float(torch.softmax(prediction, dim=1)[0][1])
#         # b = str(round(b, 3))
#         # output.value = a + '/' + b
        
#         if len(l2) < 100:   
#             l2.append((t.astype(int), 0))
#         else:
#             output.value = 'complete'
                

def close(cap):
    cap.release()
    frame_memory.close()
    frame_memory.unlink()
    contours_memory.close()
    contours_memory.unlink()
    p1.terminate()

def view(grid):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    frame_data = np.ndarray((height_img,width_img,3), dtype=np.uint8, buffer=frame_memory.buf)
    contours_data = np.ndarray((1,4), dtype=np.int32, buffer=contours_memory.buf)

    while True:
        _, frame = cap.read()

        frame_masked = preprocess(frame)
        frame_contours = contours(frame_masked, frame)

        if frame_contours.size != 0:
            img_warped = get_warp(frame, frame_contours)
            contours_size.value = 1
            if tmp.value == 1:
                frame_data[:,:,:] = img_warped[:,:,:]
                contours_data = np.ndarray((len_contours.value,4), dtype=np.int32, buffer=contours_memory.buf)
                tmp.value = 0
            frame_detect = text_detection(img_warped, contours_data)
        else:
            img_warped = frame.copy()
            len_contours.value = 0
            frame_data[:,:,:] = img_warped[0,0,0]

        img_stack = cvzone.stackImages([
            frame,
            img_warped,
        ], 2, 1)

        _, frame = cv2.imencode('.jpeg', img_stack) 
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value==True:
            close(cap)
            display_handle.update(None)

display(grid)
thread = threading.Thread(target=view, args=(grid,))
thread.start()

GridBox(children=(ToggleButton(value=False, description='Stop'), HTML(value=' ', description='Train:')), layou…

None

In [70]:
p1.terminate()
frame_memory.close()
contours_memory.close()
frame_memory.unlink()
contours_memory.unlink()
time.sleep(0.1)
p1.is_alive()

False

In [68]:
print(thread.is_alive(), p1.is_alive())

False False


## -----------------

In [7]:
l2 = []

In [43]:
len(l2)

400

In [9]:
# a = np.zeros(48).reshape(-1,4)
# a
# for _ in range(100):
#     l2.append((a.astype(int), 0))

In [62]:
l2[121]

(array([[  24,   34,  394,   77],
        [  21,   65,  399,  126],
        [  26,  198,  271,  239],
        [  23,  227,  401,  288],
        [  24,  360, 1260,  434],
        [  26,  431, 1261,  497],
        [  26,  475, 1265,  542],
        [  49,  621, 1240,  725],
        [  49,  702, 1238,  802],
        [  56,  787, 1243,  878]]),
 1)

In [13]:
def data_prepare(itemlist):
    s = 0
    # len_dict = len(target_dict)
    len_dict = 2
    
    for i in itemlist:
        if s < i[0].shape[0]:
            s = i[0].shape[0]
            
    ret_input = np.empty((1, s * 4))
    # ret_target = np.empty((1, len_dict))
    # ret_target = np.empty((1, 1))
    # ret_target = np.empty((1))
    ret_target = []
    eye = np.eye(len_dict)
            
    for i in itemlist:
        t = np.pad(i[0], [(0, s - i[0].shape[0]),(0, 0)], mode='constant')
        
        ret_input = np.vstack((ret_input, t.T.reshape(1, -1)))
        # ret_target = np.vstack((ret_target , eye[i[1]]))
        # ret_target = np.vstack((ret_target , i[1]))
        ret_target.append(i[1])

        
    # return ret_input[1:,:], ret_target[1:,:], s
    return ret_input[1:,:], ret_target, s

In [14]:
pr = data_prepare(l2)

In [63]:
pr[0][121]

array([  24.,   21.,   26.,   23.,   24.,   26.,   26.,   49.,   49.,
         56.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,   34.,   65.,  198.,  227.,  360.,  431.,  475.,  621.,
        702.,  787.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,  394.,  399.,  271.,  401., 1260., 1261., 1265.,
       1240., 1238., 1243.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,

In [45]:
X_train, X_test, y_train, y_test = train_test_split(pr[0], pr[1], test_size=0.2)

In [47]:
X_train.shape, X_test.shape, len(y_train), len(y_test)

((320, 184), (80, 184), 320, 80)

In [ ]:
for i in pr[0]:
    print(i.shape)

In [14]:
pr[0][0]

array([  18.,   19.,   24.,   21.,   23.,   25.,   24.,   47.,   49.,
         51.,    0.,    0.,   34.,   68.,  201.,  231.,  361.,  428.,
        474.,  621.,  703.,  781.,    0.,    0.,  394.,  397.,  270.,
        399., 1258., 1260., 1261., 1235., 1232., 1240.,    0.,    0.,
         82.,  130.,  243.,  294.,  439.,  503.,  548.,  729.,  805.,
        883.,    0.,    0.])

## ---------------------------------

In [69]:
# class MyNet(nn.Module):
#     def __init__(self):
#         super(MyNet, self).__init__()
#         self.flatten = nn.Flatten()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(200, 1024),
#             nn.ReLU(),
#             nn.Linear(24, 12),
#             nn.ReLU(),
#             nn.Linear(12, 6),
#             nn.ReLU(),
#             nn.Linear(6, 1),
#         )

#     def forward(self, x):
#         x = self.flatten(x)
#         logits = self.linear_relu_stack(x)
#         return logits

In [15]:
class Model_ID(nn.Module):
    def __init__(self, in_layers, out_layers):
        super(Model_ID, self).__init__()
        self.start = nn.Sequential(
            nn.Linear(in_layers, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
        )
        self.hidden = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
        )
        self.out = nn.Linear(512, out_layers)

    def forward(self, x):
        x = self.start(x)
        x = self.hidden(x)
        x = self.out(x)
        # return torch.softmax(x, dim=1)
        return x

In [71]:
pr[0].shape[1]

184

In [16]:
model = Model_ID(pr[0].shape[1], 4)

In [17]:
model

Model_ID(
  (start): Sequential(
    (0): Linear(in_features=184, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (hidden): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (out): Linear(in_features=512, out_features=4, bias=True)
)

In [18]:
class MyDataset(Dataset):
    def __init__(self, X, y, sh):
        # self.X = torch.tensor(X.astype(np.float32))
        # self.y = torch.tensor(np.array(y).astype(np.float32))
        self.X = torch.tensor(X.astype(np.float32))
        self.y = torch.tensor(np.array(y)).type(torch.LongTensor)

    def __getitem__(self, x):
        # im = self.X[x]
        # im = np.reshape(im, (1, 1, sh))
        return self.X[x].to(device), self.y[x].to(device)

    def __len__(self):
        return self.X.shape[0]

In [49]:
train_data = MyDataset(X_train, y_train, pr[0].shape[1])
train_dataloader = DataLoader(train_data, batch_size=1, shuffle=True)
test_data = MyDataset(X_test, y_test, pr[0].shape[1])
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=True)

In [50]:
model_dataloader.dataset.y[320]

tensor(3)

In [77]:
def train_batch(x, y, model, opt, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    is_correct = (prediction.argmax(1) == y)
    return is_correct.cpu().numpy().tolist()

def val_batch(x, y, model, opt, loss_fn):
    model.eval()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    return batch_loss.item()

model = Model_ID(pr[0].shape[1],4).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-5)

train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
for epoch in range(2):
    print(f"   epoch {epoch + 1}")
    train_epoch_losses, train_epoch_accuracies = [], []
    val_epoch_losses, val_epoch_accuracies = [], []

    for ix, batch in enumerate(train_dataloader):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)
        train_epoch_losses.append(batch_loss)
    train_epoch_loss = np.array(train_epoch_losses).mean()

    for ix, batch in enumerate(iter(train_dataloader)):
        x, y = batch
        is_correct = accuracy(x, y, model)
        train_epoch_accuracies.extend(is_correct)
    train_epoch_accuracy = np.mean(train_epoch_accuracies)
    
    for ix, batch in enumerate(iter(test_dataloader)):
        x, y = batch
    
        batch_loss = val_batch(x, y, model, optimizer, loss_fn)
        val_epoch_losses.append(batch_loss)
        
        val_is_correct = accuracy(x, y, model)
        val_epoch_accuracies.extend(val_is_correct)

    # for ix, batch in enumerate(model_dataloader):
    #     x, y = batch

    #     batch_loss = val_batch(x, y, model, optimizer, loss_fn)
    #     val_epoch_losses.append(batch_loss)
        
    #     is_correct = accuracy(x, y, model)
    #     train_epoch_accuracies.extend(is_correct)
    # train_epoch_accuracy = np.mean(train_epoch_accuracies)

    val_epoch_loss = np.array(val_epoch_losses).mean()
    val_epoch_accuracy = np.mean(val_epoch_accuracies)
    
    train_losses.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracies.append(val_epoch_accuracy)

    print('tr_loss',train_epoch_loss)
    print('tr_acc',train_epoch_accuracy)
    print('val_loss',val_epoch_loss)
    print('val_acc',val_epoch_accuracy)

   epoch 1
tr_loss 35.839354817413174
tr_acc 0.990625
val_loss 0.7127502048388121
val_acc 0.9625
   epoch 2
tr_loss 11.476247092783378
tr_acc 1.0
val_loss 0.00036164413900223734
val_acc 1.0


In [78]:
torch.save(model.state_dict(), 'temp_model_0.pth')

## -------------------------------

In [79]:
model = Model_ID(pr[0].shape[1],4).to(device)
model.load_state_dict(torch.load('temp_model_0.pth', map_location=torch.device('cpu')))
# model = model.load_state_dict(torch.load('temp_model_0.pth'))
model.eval()

Model_ID(
  (start): Sequential(
    (0): Linear(in_features=184, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (hidden): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (out): Linear(in_features=512, out_features=4, bias=True)
)

In [80]:
# x = pr[0][0]
x = np.zeros(48)
x = pr[0][121]
print(x)
x = torch.tensor(np.reshape(x, (1,x.shape[0]))).float().to(device)
print(x)
prediction = model(x)
torch.softmax(prediction, dim=1)[0]

[  24.   21.   26.   23.   24.   26.   26.   49.   49.   56.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   34.   65.
  198.  227.  360.  431.  475.  621.  702.  787.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.  394.  399.  271.  401.
 1260. 1261. 1265. 1240. 1238. 1243.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.   77.  126.  239.  288.  434.  497.
  542.  725.  802.  878.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.   

tensor([8.6906e-41, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       grad_fn=<SelectBackward0>)

In [119]:
float(torch.softmax(prediction, dim=1)[0][1])

0.3162335157394409

In [184]:
pr[0][0]

array([  41.,   40.,   38.,   35.,   28.,   27.,   26., 1110.,   42.,
         39.,   41.,    0.,   15.,   50.,  178.,  212.,  359.,  426.,
        472.,  499.,  624.,  701.,  784.,    0.,  416.,  418.,  285.,
        414., 1265., 1262., 1114., 1261., 1231., 1227., 1228.,    0.,
         56.,  102.,  215.,  268.,  443.,  507.,  547.,  550.,  729.,
        810.,  887.,    0.])

In [118]:
np.zeros(52)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

## -----------------------------

In [8]:
model.start = nn.Linear(500, 1024)

In [180]:
device

'mps'